# ER Wait Time File Data Cleaning

This notebook is used to clean the raw ER wait time dataset.

### Information about the raw ER Wait Time dataset and Source of the Data:
* The raw ER wait time dataset being handled within this notebook and used throughout the project **is the property of the Canadian Broadcasting Corporation (CBC)**.
    * Alberta Health Services is the primary source of the data, and CBC is the aggregator of the data.
    * The instantaneous wait times for each hospital in Alberta is at the follow Alberta Health Services website:  https://www.albertahealthservices.ca/waittimes/Page14230.aspx
    * CBC has aggregated the AHS data by recording the instantaneous wait time one per hour for all available hospitals from the AHS website.
    * The CBC reports on trends at each major city's ERs over the past 4 weeks here:
        * Calgary:  https://www.cbc.ca/news/canada/calgary/calgary-er-wait-time-tracker-cbc-1.6701714
        * Edmonton:  https://www.cbc.ca/news/canada/edmonton/edmonton-er-wait-time-tracker-cbc-1.6708480
    * The raw data being handled here and in this project is the full dataset underlying the news reports above, and has been provided courtesy of the CBC for the purposes of this data science bootcamp project.
        * As agreed with the CBC, the raw dataset will **not** be made available publically and **therefore this GitHub site does not include the raw .csv file in the `data` directory.**
* CBC's Journalistic Standards and Practises can be found here.  Expand the `Data Journalism` dropdown for pertinent information.
    * https://cbc.radio-canada.ca/en/vision/governance/journalistic-standards-and-practices/sources


### What is done in this notebook:
The raw ER data file provided by the CBC:
* is formatted for Python `datetime` objects
* is "melted" so that Hospital becomes a column
* is cleaned (null values dropped)
* is merged with my Hospitals Info dataframe to provide additional features ('city', 'services', 'citytype') for models to train on
* has feature engineering done to add a 'dayperiod', and separate the 'datetime' into 'weekofyear' and 'dayofweek' and 'hour'

In [2]:
from modules.utility import Utility
from IPython.display import display
import pandas as pd
import numpy as np
from datetime import datetime

In [30]:
# # Uncomment code as required to force reload utility.py
# # Was only required during development of this notebook

# from importlib import reload
# import modules.utility
# reload(modules.utility)
# from modules.utility import Utility

# Setup

Remove display limits for column and row lengths, for easier viewing of dataframes

In [3]:
# Set no limits for number of columns and rows to display
# Utility.reset_display_max_cols_rows() to reset
Utility.setup_display_max_cols_rows()

Load the raw wait data

In [4]:
raw = Utility.get_raw_waittimes_dataframe()

Load Cleaned Hospital Info data to do joins to hospital wait time data later

In [5]:
info = Utility.get_hospital_info_dataframe()

# Create 'raw' df
* Load in the .csv version of CBC's screen-scraped data with no changes in Excel

In [7]:
raw.head()

,Date,Year,Time,Alberta Children's Hospital,Foothills Medical Centre,Peter Lougheed Centre,Rockyview General Hospital,South Health Campus,Sheldon M. Chumir Centre,South Calgary Health Centre,Airdrie Community Health Centre,Cochrane Community Health Centre,Okotoks Health and Wellness Centre,Devon General Hospital,Grey Nuns Community Hospital,Misericordia Community Hospital,Royal Alexandra Hospital,Strathcona Community Hospital,University of Alberta Hospital,Fort Sask Community Hospital,Leduc Community Hospital,Northeast Community Health Centre,Stollery Children's Hospital,Sturgeon Community Hospital,WestView Health Centre,Chinook Regional Hospital,Medicine Hat Regional Hospital,Red Deer Regional Hospital,Lacombe Hospital and Care Centre,Innisfail Health Centre
0,24-Sep,2022,6:22 PM,201.0,188.0,116.0,148.0,154.0,89.0,193.0,NaN,NaN,NaN,132.0,147.0,384.0,287.0,196.0,234.0,295.0,293.0,135.0,199.0,261.0,189.0,69.0,96.0,161.0,NaN,NaN
1,24-Sep,2022,6:23 PM,201.0,188.0,116.0,148.0,154.0,89.0,193.0,NaN,NaN,NaN,132.0,147.0,384.0,287.0,196.0,234.0,295.0,293.0,135.0,199.0,261.0,189.0,69.0,96.0,161.0,NaN,NaN
2,24-Sep,2022,8:25 PM,236.0,140.0,124.0,120.0,150.0,58.0,NaN,NaN,NaN,NaN,158.0,224.0,285.0,415.0,251.0,301.0,188.0,158.0,283.0,179.0,203.0,351.0,79.0,60.0,241.0,NaN,NaN
3,24-Sep,2022,8:27 PM,236.0,140.0,124.0,120.0,150.0,58.0,NaN,NaN,NaN,NaN,158.0,224.0,285.0,415.0,251.0,301.0,188.0,158.0,283.0,179.0,203.0,351.0,79.0,60.0,241.0,NaN,NaN
4,24-Sep,2022,8:28 PM,237.0,140.0,124.0,106.0,151.0,58.0,NaN,NaN,NaN,NaN,158.0,216.0,281.0,415.0,251.0,301.0,188.0,158.0,283.0,179.0,203.0,351.0,73.0,60.0,230.0,NaN,NaN


In [8]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9186 entries, 0 to 9185
Data columns (total 30 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Date                                9186 non-null   object 
 1   Year                                9186 non-null   int64  
 2   Time                                9186 non-null   object 
 3   Alberta Children's Hospital         9111 non-null   float64
 4   Foothills Medical Centre            9093 non-null   float64
 5   Peter Lougheed Centre               9103 non-null   float64
 6   Rockyview General Hospital          9108 non-null   float64
 7   South Health Campus                 9108 non-null   float64
 8   Sheldon M. Chumir Centre            9111 non-null   float64
 9   South Calgary Health Centre         4556 non-null   float64
 10  Airdrie Community Health Centre     5153 non-null   float64
 11  Cochrane Community Health Centre    405 non

## Make 'datetime' column from 'Date', 'Year', 'Time' columns in raw dataset

In [10]:
df = raw.copy()
Utility.make_datetime_column_on_raw_waittime_df(df)
#df.insert(df.columns.get_loc('Time') + 1, 'datetime', pd.to_datetime(df['Year'].astype(str) + ' ' + df['Date'] + ' ' + df['Time'], format='%Y %d-%b %I:%M %p'))

In [11]:
df.head()

,Date,Year,Time,datetime,Alberta Children's Hospital,Foothills Medical Centre,Peter Lougheed Centre,Rockyview General Hospital,South Health Campus,Sheldon M. Chumir Centre,South Calgary Health Centre,Airdrie Community Health Centre,Cochrane Community Health Centre,Okotoks Health and Wellness Centre,Devon General Hospital,Grey Nuns Community Hospital,Misericordia Community Hospital,Royal Alexandra Hospital,Strathcona Community Hospital,University of Alberta Hospital,Fort Sask Community Hospital,Leduc Community Hospital,Northeast Community Health Centre,Stollery Children's Hospital,Sturgeon Community Hospital,WestView Health Centre,Chinook Regional Hospital,Medicine Hat Regional Hospital,Red Deer Regional Hospital,Lacombe Hospital and Care Centre,Innisfail Health Centre
0,24-Sep,2022,6:22 PM,2022-09-24 18:22:00,201.0,188.0,116.0,148.0,154.0,89.0,193.0,NaN,NaN,NaN,132.0,147.0,384.0,287.0,196.0,234.0,295.0,293.0,135.0,199.0,261.0,189.0,69.0,96.0,161.0,NaN,NaN
1,24-Sep,2022,6:23 PM,2022-09-24 18:23:00,201.0,188.0,116.0,148.0,154.0,89.0,193.0,NaN,NaN,NaN,132.0,147.0,384.0,287.0,196.0,234.0,295.0,293.0,135.0,199.0,261.0,189.0,69.0,96.0,161.0,NaN,NaN
2,24-Sep,2022,8:25 PM,2022-09-24 20:25:00,236.0,140.0,124.0,120.0,150.0,58.0,NaN,NaN,NaN,NaN,158.0,224.0,285.0,415.0,251.0,301.0,188.0,158.0,283.0,179.0,203.0,351.0,79.0,60.0,241.0,NaN,NaN
3,24-Sep,2022,8:27 PM,2022-09-24 20:27:00,236.0,140.0,124.0,120.0,150.0,58.0,NaN,NaN,NaN,NaN,158.0,224.0,285.0,415.0,251.0,301.0,188.0,158.0,283.0,179.0,203.0,351.0,79.0,60.0,241.0,NaN,NaN
4,24-Sep,2022,8:28 PM,2022-09-24 20:28:00,237.0,140.0,124.0,106.0,151.0,58.0,NaN,NaN,NaN,NaN,158.0,216.0,281.0,415.0,251.0,301.0,188.0,158.0,283.0,179.0,203.0,351.0,73.0,60.0,230.0,NaN,NaN


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9186 entries, 0 to 9185
Data columns (total 31 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   Date                                9186 non-null   object        
 1   Year                                9186 non-null   int64         
 2   Time                                9186 non-null   object        
 3   datetime                            9186 non-null   datetime64[ns]
 4   Alberta Children's Hospital         9111 non-null   float64       
 5   Foothills Medical Centre            9093 non-null   float64       
 6   Peter Lougheed Centre               9103 non-null   float64       
 7   Rockyview General Hospital          9108 non-null   float64       
 8   South Health Campus                 9108 non-null   float64       
 9   Sheldon M. Chumir Centre            9111 non-null   float64       
 10  South Calgary Health Cen

Descriptive Statistics on original (numeric) data (minus 'Year')

In [13]:
# Show descriptive statistics for all numerical columns except 'Year'
df.drop(['Year'], axis=1).describe()

,datetime,Alberta Children's Hospital,Foothills Medical Centre,Peter Lougheed Centre,Rockyview General Hospital,South Health Campus,Sheldon M. Chumir Centre,South Calgary Health Centre,Airdrie Community Health Centre,Cochrane Community Health Centre,Okotoks Health and Wellness Centre,Devon General Hospital,Grey Nuns Community Hospital,Misericordia Community Hospital,Royal Alexandra Hospital,Strathcona Community Hospital,University of Alberta Hospital,Fort Sask Community Hospital,Leduc Community Hospital,Northeast Community Health Centre,Stollery Children's Hospital,Sturgeon Community Hospital,WestView Health Centre,Chinook Regional Hospital,Medicine Hat Regional Hospital,Red Deer Regional Hospital,Lacombe Hospital and Care Centre,Innisfail Health Centre
count,9186,9111.000000,9093.000000,9103.000000,9108.000000,9108.000000,9111.000000,4556.000000,5153.000000,405.000000,0.0,9139.000000,9138.000000,9131.000000,9138.000000,9139.000000,9138.000000,9138.000000,9138.000000,9139.000000,9138.000000,9138.000000,9138.000000,8822.000000,8816.000000,9113.000000,0.0,0.0
mean,2023-04-24 07:19:15.558458880,199.572385,247.315188,295.444908,223.920290,281.944664,214.177917,128.221686,154.160295,98.664198,NaN,137.568881,180.614467,289.894316,273.938499,174.418536,270.942766,175.356095,151.941125,209.492395,193.127708,202.523309,181.231889,102.804126,92.503063,258.548776,NaN,NaN
min,2022-09-24 18:22:00,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,NaN,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,NaN,NaN
25%,2023-01-11 02:16:00,115.000000,182.000000,182.000000,148.000000,189.000000,126.000000,92.000000,95.000000,45.000000,NaN,55.000000,129.000000,208.000000,206.000000,107.000000,194.000000,87.000000,79.000000,140.000000,118.000000,132.250000,108.000000,52.000000,42.000000,162.000000,NaN,NaN
50%,2023-04-17 00:31:00,170.000000,235.000000,263.000000,198.000000,250.000000,181.000000,126.000000,141.000000,90.000000,NaN,123.000000,173.000000,277.000000,268.000000,170.000000,265.000000,160.000000,140.000000,197.000000,179.000000,196.000000,174.000000,89.000000,81.000000,235.000000,NaN,NaN
75%,2023-08-18 19:46:00,247.000000,297.000000,373.000000,269.000000,341.000000,273.000000,165.000000,198.000000,135.000000,NaN,201.000000,222.000000,357.500000,336.000000,230.000000,340.000000,250.000000,212.000000,261.000000,246.000000,266.000000,243.000000,136.000000,126.250000,331.000000,NaN,NaN
max,2023-11-22 14:01:00,1036.000000,775.000000,1175.000000,827.000000,962.000000,873.000000,356.000000,595.000000,417.000000,NaN,663.000000,600.000000,873.000000,623.000000,667.000000,732.000000,863.000000,548.000000,916.000000,988.000000,723.000000,658.000000,593.000000,443.000000,1069.000000,NaN,NaN
std,NaN,129.254659,94.868087,158.920631,110.241707,136.238886,124.172176,53.689182,84.232355,62.620465,NaN,90.998597,79.754128,116.844342,96.761785,91.012240,103.040386,114.524353,91.975606,100.464304,109.721164,97.699090,97.816309,65.807900,59.517961,136.640915,NaN,NaN


In [14]:
df.columns

Index(['Date', 'Year', 'Time', 'datetime', 'Alberta Children's Hospital',
       'Foothills Medical Centre', 'Peter Lougheed Centre',
       'Rockyview General Hospital', 'South Health Campus',
       'Sheldon M. Chumir Centre', 'South Calgary Health Centre',
       'Airdrie Community Health Centre', 'Cochrane Community Health Centre',
       'Okotoks Health and Wellness Centre', 'Devon General Hospital',
       'Grey Nuns Community Hospital', 'Misericordia Community Hospital',
       'Royal Alexandra Hospital', 'Strathcona Community Hospital',
       'University of Alberta Hospital', 'Fort Sask Community Hospital',
       'Leduc Community Hospital', 'Northeast Community Health Centre',
       'Stollery Children's Hospital', 'Sturgeon Community Hospital',
       'WestView Health Centre', 'Chinook Regional Hospital',
       'Medicine Hat Regional Hospital', 'Red Deer Regional Hospital',
       'Lacombe Hospital and Care Centre', 'Innisfail Health Centre'],
      dtype='object')

Define function that princes stats about number of nulls for each column in a dataframe

In [16]:
# Use Utility function to print Null statistics for dataframe
Utility.print_num_stats(df)

Stats for Number of Nulls:

Column Name                             :      Null Percentage     Num Nulls      Num Rows
-----------                             :      ---------------     ---------      --------
Date                                    :                 0.00%            0         9186
Year                                    :                 0.00%            0         9186
Time                                    :                 0.00%            0         9186
datetime                                :                 0.00%            0         9186
Alberta Children's Hospital             :                 0.82%           75         9186
Foothills Medical Centre                :                 1.01%           93         9186
Peter Lougheed Centre                   :                 0.90%           83         9186
Rockyview General Hospital              :                 0.85%           78         9186
South Health Campus                     :                 0.85%       

Drop columns corresponding to hospitals that have 100% null data

In [17]:
# Since Okotoks, Lacombe and Innisfail have 100% nulls, drop these columns altogether.
df.drop(columns=['Okotoks Health and Wellness Centre', 'Lacombe Hospital and Care Centre', 'Innisfail Health Centre'], axis=1, inplace=True)

# Create `df_melted`
* Melt (reverse-pivot) the dataset so the columns are: Datetime, Hospital, Wait Time
* New dataframe will be called df_melted

In [18]:
# Melt the DF (reverse pivot) to put the Hospital Names into a single column
desired_columns = (df.columns[~df.columns.isin(['Date', 'Time', 'Year'])]).tolist()
df_melted = pd.melt(df[desired_columns], id_vars=['datetime'], var_name='hospital', value_name='waittime')

In [19]:
df_melted.head(15)

,datetime,hospital,waittime
0,2022-09-24 18:22:00,Alberta Children's Hospital,201.0
1,2022-09-24 18:23:00,Alberta Children's Hospital,201.0
2,2022-09-24 20:25:00,Alberta Children's Hospital,236.0
3,2022-09-24 20:27:00,Alberta Children's Hospital,236.0
4,2022-09-24 20:28:00,Alberta Children's Hospital,237.0
5,2022-09-24 20:29:00,Alberta Children's Hospital,239.0
6,2022-09-24 20:30:00,Alberta Children's Hospital,239.0
7,2022-09-24 20:31:00,Alberta Children's Hospital,239.0
8,2022-09-24 20:32:00,Alberta Children's Hospital,241.0
9,2022-09-24 20:33:00,Alberta Children's Hospital,241.0


On new `df_melted` dataframe: Sort ascending by datetime and reset index

In [21]:
# Sort the df_melted dataframe ascending by Datetime and Hospital

df_melted.sort_values(by=['datetime', 'hospital'], ascending=[True, True], inplace=True)
df_melted.reset_index(drop=True, inplace=True)

In [22]:
df_melted.head()

,datetime,hospital,waittime
0,2022-09-24 18:22:00,Airdrie Community Health Centre,NaN
1,2022-09-24 18:22:00,Alberta Children's Hospital,201.0
2,2022-09-24 18:22:00,Chinook Regional Hospital,69.0
3,2022-09-24 18:22:00,Cochrane Community Health Centre,NaN
4,2022-09-24 18:22:00,Devon General Hospital,132.0


In [23]:
df_melted.shape

(220464, 3)

In [24]:
Utility.print_num_stats(df_melted)

Stats for Number of Nulls:

Column Name                             :      Null Percentage     Num Nulls      Num Rows
-----------                             :      ---------------     ---------      --------
datetime                                :                 0.00%            0       220464
hospital                                :                 0.00%            0       220464
waittime                                :                 8.76%        19313       220464


Prepare to drop remaining null rows corresponding to datetime values where there was no waittime information for a given hospital

In [25]:
# Show how many rows of data, per hospital per date, will be dropped when I dropna() on this dataframe in the next cell.
# Credit to GitHub Copilot for the below code:
df_nulls = df_melted[df_melted['waittime'].isnull()][['hospital', 'datetime']].value_counts().reset_index()
df_nulls.columns = ['hospital', 'datetime', 'Count of Nulls to be Dropped']
df_nulls['datetime'] = df_nulls['datetime'].dt.date
df_nulls = df_nulls.drop_duplicates(subset=['hospital', 'datetime'], keep='first')
df_nulls.head(50)

,hospital,datetime,Count of Nulls to be Dropped
0,Sheldon M. Chumir Centre,2022-11-06,2
1,Red Deer Regional Hospital,2023-11-05,2
2,South Calgary Health Centre,2023-11-05,2
3,South Calgary Health Centre,2022-11-06,2
4,Airdrie Community Health Centre,2022-11-06,2
5,Chinook Regional Hospital,2023-11-05,2
6,Chinook Regional Hospital,2022-11-06,2
7,Cochrane Community Health Centre,2023-11-05,2
8,Cochrane Community Health Centre,2022-11-06,2
9,Medicine Hat Regional Hospital,2022-11-06,2


# Create new dataframe `df_clean` which calls dropna() on `df_melted`:
* to drop all datetime entries per hospital, with null waittime
* Note: We cannot easily (nor do we want to) impute these nulls, so will just drop them.

In [26]:
# Drop all rows with null WaitTimes
df_clean = df_melted.dropna().reset_index(drop=True)

In [27]:
df_clean.head()

,datetime,hospital,waittime
0,2022-09-24 18:22:00,Alberta Children's Hospital,201.0
1,2022-09-24 18:22:00,Chinook Regional Hospital,69.0
2,2022-09-24 18:22:00,Devon General Hospital,132.0
3,2022-09-24 18:22:00,Foothills Medical Centre,188.0
4,2022-09-24 18:22:00,Fort Sask Community Hospital,295.0


In [28]:
df_clean.shape

(201151, 3)

In [29]:
# Check our numbers - that nothing was dropped that shouldn't have been.  Expecting a "True" from the assert below:
df_melted.shape[0] - df_clean.shape[0] == df_melted[df_melted['waittime'].isnull()].shape[0]

True

# Feature Engineering on `df_clean`
Add the following columns:
* Year
* Day of Week
* Week of Year - is the isocalendar week (starting on Monday, ending on Sunday)
* Month of Year
* Hour
* Day Period:  Per an ER Clinician-Researcher, use the following:
    - 'day': 08h00-15h59
    - 'evening': 16h00-2359 evening
    - 'night': 24h00-7h59
* City
* Services: (from Hospitals Info df)
    - 'emergency': Offers 24x7 emergency care per AHS website on the hospital
    -  'urgentcare': Does not have an ER per per AHS website on the hospital
* City Status (from Hospitals Info df)
    - 'urban': Urban (if city population >= 600,000)
    - 'peri': Peri-Urban (if city population >= 30,000 and < 600,000)
    - 'rural': Rural setting (if city population < 30,000)


In [32]:
# Add the features for Year, Day of Week, Week of Year, Hour, Day Period

df_clean['year'] = df_clean['datetime'].dt.year
df_clean['dayofweek'] = df_clean['datetime'].dt.day_name()
df_clean['numdayofweek'] = df_clean['datetime'].dt.dayofweek # May later want a number 0-6, 0 = Monday, 6 = Sunday
df_clean['weekofyear'] = df_clean['datetime'].dt.isocalendar().week
df_clean['hour'] = df_clean['datetime'].dt.hour

# Create 'dayperiod' feature using definitions of bins and labels from Utility class
Utility.add_dayperiod_feature(df_clean)

In [34]:
df_clean.columns

Index(['datetime', 'hospital', 'waittime', 'year', 'dayofweek', 'numdayofweek',
       'weekofyear', 'hour', 'dayperiod'],
      dtype='object')

In [35]:
# Remind ourselves about the hospital info dataframe which we'll merged into df_clean
info.head()

,name,id,services,city,province,city.area,city.lat,city.long,city.pop,citytype,address,hosp_lat,hosp_long
0,Alberta Children's Hospital,ach,emergency,Calgary,Alberta,826,51.05,-114.0667,"1,306,784",urban,"28 Oki Drive NW, Calgary, Alberta, T3B 6A8",51.074759,-114.146833
1,Foothills Medical Centre,fmc,emergency,Calgary,Alberta,826,51.05,-114.0667,"1,306,784",urban,"1403 29 Street NW, Calgary, Alberta, T2N 2T9",51.064657,-114.130926
2,Peter Lougheed Centre,plc,emergency,Calgary,Alberta,826,51.05,-114.0667,"1,306,784",urban,"3500 26 Avenue NE, Calgary, Alberta, T1Y 6J4",51.078914,-113.984611
3,Rockyview General Hospital,rgh,emergency,Calgary,Alberta,826,51.05,-114.0667,"1,306,784",urban,"7007 14 Street SW, Calgary, Alberta, T2V 1P9",50.990056,-114.097077
4,South Health Campus,shc,emergency,Calgary,Alberta,826,51.05,-114.0667,"1,306,784",urban,"4448 Front Street SE, Calgary, Alberta, T3M 1M4",50.880825,-113.952720


In [36]:
info['id'].tolist()

['ach',
 'fmc',
 'plc',
 'rgh',
 'shc',
 'smcc',
 'schc',
 'achc',
 'cchc',
 'ohwc',
 'dgh',
 'gnch',
 'mch',
 'rah',
 'schc',
 'uah',
 'fsch',
 'lch',
 'nchc',
 'stollery',
 'sch',
 'whc',
 'crh',
 'mhrh',
 'rdrh',
 'lhcc',
 'ihc']

# Create `df_merged` dataframe:
* is `df_clean` plus city, city.area, city.pop, citytype, hospital id, hosp_lat, hosp_long features from the Hospital info (info) dataframe

In [37]:
# Add the City, Services, and CityStatus features from the hosptial_info_clean.csv file (info df)
df_merged = pd.merge(df_clean, info[['id', 'services', 'city', 'city.area', 'city.pop', 'citytype', 'hosp_lat', 'hosp_long', 'name']], how='left', left_on='hospital', right_on='name')
df_merged.drop(columns=['name'], axis=1, inplace=True)

In [38]:
df_merged.head(2)

,datetime,hospital,waittime,year,dayofweek,numdayofweek,weekofyear,hour,dayperiod,id,services,city,city.area,city.pop,citytype,hosp_lat,hosp_long
0,2022-09-24 18:22:00,Alberta Children's Hospital,201.0,2022,Saturday,5,38,18,evening,ach,emergency,Calgary,826,"1,306,784",urban,51.074759,-114.146833
1,2022-09-24 18:22:00,Chinook Regional Hospital,69.0,2022,Saturday,5,38,18,evening,crh,emergency,Lethbridge,122,"98,406",peri,49.685221,-112.816209


In [39]:
df_merged.rename(columns={
    'city.area': 'cityarea',
    'city.pop': 'citypop',
    'hosp_lat': 'hosplat',
    'hosp_long': 'hosplong'
    },
    inplace=True)

In [40]:
df_merged.head(2)

,datetime,hospital,waittime,year,dayofweek,numdayofweek,weekofyear,hour,dayperiod,id,services,city,cityarea,citypop,citytype,hosplat,hosplong
0,2022-09-24 18:22:00,Alberta Children's Hospital,201.0,2022,Saturday,5,38,18,evening,ach,emergency,Calgary,826,"1,306,784",urban,51.074759,-114.146833
1,2022-09-24 18:22:00,Chinook Regional Hospital,69.0,2022,Saturday,5,38,18,evening,crh,emergency,Lethbridge,122,"98,406",peri,49.685221,-112.816209


Statistics of row counts (number of waittime observations) for each hospital after the null rows were dropped

In [41]:
# Get the counts for each hospital
df_merged.groupby(['city', 'hospital']).size().sort_index(level=[0, 1], sort_remaining=False).sort_values(ascending=True)

city               hospital                         
Cochrane           Cochrane Community Health Centre      405
Calgary            South Calgary Health Centre          4556
Airdrie            Airdrie Community Health Centre      5153
Medicine Hat       Medicine Hat Regional Hospital       8816
Lethbridge         Chinook Regional Hospital            8822
Calgary            Foothills Medical Centre             9093
                   Peter Lougheed Centre                9103
                   Rockyview General Hospital           9108
                   South Health Campus                  9108
                   Alberta Children's Hospital          9111
                   Sheldon M. Chumir Centre             9111
Red Deer           Red Deer Regional Hospital           9113
Edmonton           Misericordia Community Hospital      9131
                   Royal Alexandra Hospital             9138
                   Stollery Children's Hospital         9138
                   University of

Check datatypes to ensure all numerical data is cast as integers

In [42]:
df_merged.dtypes

datetime        datetime64[ns]
hospital                object
waittime               float64
year                     int32
dayofweek               object
numdayofweek             int32
weekofyear              UInt32
hour                     int32
dayperiod             category
id                      object
services                object
city                    object
cityarea                 int64
citypop                 object
citytype                object
hosplat                float64
hosplong               float64
dtype: object

In [43]:
# Fix datatype of citypop to make it a number
df_merged['citypop'] = df_merged['citypop'].str.replace(',', '').astype('int64')

In [44]:
df_merged.head(5)

,datetime,hospital,waittime,year,dayofweek,numdayofweek,weekofyear,hour,dayperiod,id,services,city,cityarea,citypop,citytype,hosplat,hosplong
0,2022-09-24 18:22:00,Alberta Children's Hospital,201.0,2022,Saturday,5,38,18,evening,ach,emergency,Calgary,826,1306784,urban,51.074759,-114.146833
1,2022-09-24 18:22:00,Chinook Regional Hospital,69.0,2022,Saturday,5,38,18,evening,crh,emergency,Lethbridge,122,98406,peri,49.685221,-112.816209
2,2022-09-24 18:22:00,Devon General Hospital,132.0,2022,Saturday,5,38,18,evening,dgh,emergency,Devon,14,6545,rural,53.352265,-113.728288
3,2022-09-24 18:22:00,Foothills Medical Centre,188.0,2022,Saturday,5,38,18,evening,fmc,emergency,Calgary,826,1306784,urban,51.064657,-114.130926
4,2022-09-24 18:22:00,Fort Sask Community Hospital,295.0,2022,Saturday,5,38,18,evening,fsch,emergency,Fort Saskatchewan,48,27088,rural,53.693175,-113.213436


# Write dataframes out to disk under the cleandata subdirectory:

 | Name of df in this   notebook | Description of What DataFrame   Contains                                                                                                                                                                                                                                                                                                                                                 | Path on Disk                         | Filename                               |
|-------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|--------------------------------------|----------------------------------------|
| `info`                        | Dataframe with hospital   information, to be merged with wait time dataframe.                                                                                                                                                                                                                                                                                                            | <project_root>/data/cleandata/ | hospital_info_clean.csv                |
| `df`                          | Dataframe with raw data   originally from CBC screenscrape plus extra 'datetime' column, created from   Day, Year, and Time columns from original dataset.  3 Hospital columns entirely of null values   dropped (Okotoks, Lacombe, Innisfail).                                                                                                                                          | <project_root>/data/cleandata/ | df_er_wait_time_nov_22_2023.csv        |
| `df_melted`                   | `df` dataframe with raw data and   datetime column minus 3 null value hospitals melted to be 3 columns:  'datetime', 'hospital', 'waittime'                                                                                                                                                                                                                                              | <project_root>/data/cleandata/ | df_melted_er_wait_time_nov_22_2023.csv |
| `df_clean`                    | `df_melted` dataframe with   'datetime', 'hospital', 'waittime', with null rows dropped and the following   features engineered: 'year', 'dayofweek', 'numdayofweek' (0 = Monday, 6 =   Sunday), 'weekofyear' (isocalendar with Monday start of week and Sunday end of   week), 'hour', 'dayperiod' ('night', 'day', 'evening')                                                          | <project_root>/data/cleandata/ | df_clean_er_wait_time_nov_22_2023.csv  |
| `df_merged`                   | `df_clean` dataframe with   additional columns (hospital information) merged from `info` dataframe, as   follows: 'id' ('ach', 'crh', etc.), 'services' ('emergency', 'urgentcare'),   city, cityarea, citypop, citytype ('urban', 'peri', 'rural'), hosplat, hosplong.            FINAL DF to be loaded and used in subsequent notebooks for EDA, hypothesis   testing, model building. | <project_root>/data/cleandata/ | df_final_er_wait_time_nov_22_2023.csv  |

In [55]:
# Write the raw data originally from CBC, with datetime added, and 3 null columns/hospitals dropped, to a csv file for later use

#df.to_csv('cleandata/df_er_wait_time_nov_22_2023.csv', index=False)
df.to_csv(Utility.RAW_WAIT_DATETIME_DROP_NULL_HOSPITALS_DF_FILENAME, index=False)

# Write the melted dataframe: 'datetime', 'hospital', 'waittime', merged dataframe to a csv file for later use
#df_melted.to_csv('cleandata/df_melted_er_wait_time_nov_22_2023.csv', index=False)
df_melted.to_csv(Utility.RAW_WAIT_INITIAL_MELTED_DF_MELTED_FILENAME, index=False)

# Write the cleaned dataframe: melted, nulls dropped, datetime-type features engineered, to a csv file for later use
#df_clean.to_csv('cleandata/df_clean_er_wait_time_nov_22_2023.csv', index=False)
df_clean.to_csv(Utility.RAW_WAIT_DROPNULL_DATETIME_FEATURE_ENG_DF_CLEAN_FILENAME, index=False)

# Write the final df with: datetime/hospital/waittime columns, cleaned (no nulls), feature engineered, merged dataframe, to a csv file for later use
#df_merged.to_csv('cleandata/df_final_er_wait_time_nov_22_2023.csv', index=False)
df_merged.to_csv(Utility.CLEAN_WAIT_FILENAME, index=False)